# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [72]:
# Imports
import os
import warnings
import tools
import models as c_models
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"

# Constant frame count.
frames = 100

### Preparation Stage
### Load data and normalize
For training it's required to extend/reduce every dataset to n frames, where n is `frames`.

In [62]:
listfile = os.listdir(dirname)
contents = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        contents.append((wordname, content))
data = contents

In [63]:
contents[0][1]

,face_x,face_y,landmark_x_1,landmark_y_1,landmark_x_2,landmark_y_2,landmark_x_3,landmark_y_3,landmark_x_4,landmark_y_4,...,landmark_x_38,landmark_y_38,landmark_x_39,landmark_y_39,landmark_x_40,landmark_y_40,landmark_x_41,landmark_y_41,landmark_x_42,landmark_y_42
0,0.513086,0.317546,0.734630,0.730928,0.682608,0.740892,0.647388,0.753325,0.608517,0.790681,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.514772,0.319561,0.287116,0.713952,0.332434,0.729370,0.377520,0.710072,0.398559,0.688866,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.509703,0.322235,0.286905,0.676018,0.331073,0.678926,0.375002,0.663713,0.402209,0.645312,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.511009,0.322432,0.300466,0.639038,0.346397,0.632383,0.380598,0.614600,0.408101,0.596380,...,0.303501,0.466568,0.271055,0.517031,0.258359,0.506262,0.254735,0.508821,0.256895,0.50613
4,0.511274,0.322397,0.317917,0.615976,0.366193,0.604277,0.401965,0.582840,0.425050,0.563845,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
97,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


### Split data
Split the dataset up into the following segments:
1. Training Data: 60%
2. Validation Data: 20%
3. Test Data: 20%


In [65]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

In [66]:
# Display train data
print("Total:", len(labels))
print("Training:", len(y_train), len(y_train) / len(labels) * 100)
print("Val:", len(y_val), len(y_val) / len(labels) * 100)
print("Test:", len(y_test), len(y_test) / len(labels) * 100)

Total: 347
Training: 208 59.94236311239193
Val: 69 19.88472622478386
Test: 70 20.172910662824208


### Tokenize (One Hot)

In [67]:
import tools
tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]
y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)
print(y_train)

{'computer': 1, 'deutschland': 2, 'hallo': 3, 'welt': 4}
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [68]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [55]:
print(y_train)

[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


## Training Stage

In [74]:
model = Sequential()
model.add(layers.LSTM(64, return_sequences=True,
               input_shape=(100, 86)))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [75]:
history=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_val,y_val))

Train on 208 samples, validate on 69 samples
Epoch 1/100
208/208 [==============================] - 3s 12ms/sample - loss: 1.5424 - acc: 0.3269 - precision_1: 0.5714 - recall: 0.0192 - val_loss: 1.4563 - val_acc: 0.2464 - val_precision_1: 0.8571 - val_recall: 0.1739
Epoch 2/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.3722 - acc: 0.4183 - precision_1: 0.8571 - recall: 0.1442 - val_loss: 1.3186 - val_acc: 0.3623 - val_precision_1: 0.7647 - val_recall: 0.1884
Epoch 3/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.2173 - acc: 0.4615 - precision_1: 0.9070 - recall: 0.1875 - val_loss: 1.2096 - val_acc: 0.4058 - val_precision_1: 0.7647 - val_recall: 0.1884
Epoch 4/100
208/208 [==============================] - 1s 4ms/sample - loss: 1.1538 - acc: 0.4663 - precision_1: 0.9091 - recall: 0.1923 - val_loss: 1.1509 - val_acc: 0.4058 - val_precision_1: 0.8235 - val_recall: 0.2029
Epoch 5/100
208/208 [==============================] - 1s 4ms/sample -

Epoch 38/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.5582 - acc: 0.8029 - precision_1: 0.8029 - recall: 0.8029 - val_loss: 1.0667 - val_acc: 0.5797 - val_precision_1: 0.5797 - val_recall: 0.5797
Epoch 39/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.6482 - acc: 0.7692 - precision_1: 0.7794 - recall: 0.7644 - val_loss: 0.4780 - val_acc: 0.8841 - val_precision_1: 0.8841 - val_recall: 0.8841
Epoch 40/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.5394 - acc: 0.8125 - precision_1: 0.8276 - recall: 0.8077 - val_loss: 0.5333 - val_acc: 0.8841 - val_precision_1: 0.8841 - val_recall: 0.8841
Epoch 41/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.5409 - acc: 0.8125 - precision_1: 0.8284 - recall: 0.8125 - val_loss: 0.5047 - val_acc: 0.8696 - val_precision_1: 0.8657 - val_recall: 0.8406
Epoch 42/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.4543 - acc: 0.8606 - precision_1

208/208 [==============================] - 1s 4ms/sample - loss: 0.4768 - acc: 0.8606 - precision_1: 0.8606 - recall: 0.8606 - val_loss: 0.4245 - val_acc: 0.9130 - val_precision_1: 0.9130 - val_recall: 0.9130
Epoch 75/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.5989 - acc: 0.7692 - precision_1: 0.7729 - recall: 0.7692 - val_loss: 0.4203 - val_acc: 0.9130 - val_precision_1: 0.9130 - val_recall: 0.9130
Epoch 76/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.3676 - acc: 0.8942 - precision_1: 0.8942 - recall: 0.8942 - val_loss: 0.6084 - val_acc: 0.7971 - val_precision_1: 0.7971 - val_recall: 0.7971
Epoch 77/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.4055 - acc: 0.8606 - precision_1: 0.8599 - recall: 0.8558 - val_loss: 0.3387 - val_acc: 0.9420 - val_precision_1: 0.9420 - val_recall: 0.9420
Epoch 78/100
208/208 [==============================] - 1s 4ms/sample - loss: 0.3851 - acc: 0.8846 - precision_1: 0.8922 - re

In [78]:
model.evaluate(x=x_test, y=y_test, batch_size=32)

70/70 [==============================] - 0s 1ms/sample - loss: 0.7570 - acc: 0.7714 - precision_1: 0.7714 - recall: 0.7714


[0.7569688098771231, 0.7714286, 0.7714286, 0.7714286]